# Analyse de données Budget et Finances

## Paramétrage

In [ ]:
import os

import pandas as pd
CHEMIN_DOSSIER_DONNÉES = '../../../data_test/diamant/'
DATE_FICHIER = '2022_09_01'
encoding = 'utf-8'


In [ ]:
ann_errd_ej_et = pd.read_csv(
    os.path.join(CHEMIN_DOSSIER_DONNÉES, 'ANN_ERRD_EJ_ET_' + DATE_FICHIER + '.CSV'),
    delimiter=";",
    encoding=encoding,
    usecols=['Finess EJ',
             'Finess',
             'Année',
             'Dépenses Groupe I ERRD',
             'Dépenses Groupe II ERRD',
             'Dépenses Groupe III ERRD',
             'Recettes Groupe I ERRD',
             'Recettes Groupe II ERRD',
             'Recettes Groupe III ERRD',
             'MS Résultat net comptable ERRD',
             '655 Quotes-parts de résultat sur opérations faites en commun'],
)

In [ ]:
ann_ca_ej_et = pd.read_csv(
    os.path.join(CHEMIN_DOSSIER_DONNÉES, 'ANN_CA_EJ_ET_' + DATE_FICHIER + '.CSV'),
    delimiter=";",
    encoding=encoding,
    usecols=['Finess EJ',
             'Finess',
             'Année',
             'Recettes Groupe I CA',
             'Recettes Groupe II CA',
             'Recettes Groupe III CA',
             'Dépenses Groupe I CA',
             'Dépenses Groupe II CA',
             'Dépenses Groupe III CA',
             'Charges CA PA',
             'Produits CA PA',
             'Taux de CAF CA PH',
             'Taux vétusté Construction CA',
             'MS Résultat net comptable CA PH',
             'MS Résultat net comptable CA PA'],
)

In [ ]:
ann_errd_ej = pd.read_csv(
    os.path.join(CHEMIN_DOSSIER_DONNÉES, 'ANN_ERRD_EJ_' + DATE_FICHIER + '.CSV'),
    delimiter=";",
    encoding=encoding,
    usecols=['Finess EJ',
            'Année',
            'Taux vétusté Construction ERRD',
             'Taux de CAF ERRD'],
)


## Doublons

### On s'attend à avoir une seule valeur par (Finess, Année)

In [ ]:
a_plus_d_une_donnée_par_finess_par_année = (ann_errd_ej_et.groupby(['Finess', 'Année']).nunique() > 1).any(axis=1)

## Valeurs manquantes


In [ ]:
ann_errd_ej_et.isna().sum() / ann_errd_ej_et.shape[0]


### Années renseignées

In [ ]:
ann_errd_ej_et.groupby('Finess').count()['Année'].hist()


In [ ]:
ann_errd_ej_et['Année'].nunique()

In [ ]:
ann_errd_ej_et['Année'].hist()

In [ ]:
ann_errd_ej_et['Année'].max()

### Valeurs manquantes par colonne

In [ ]:
nombre_d_établissements_dans_ann_ms_tdp_et = ann_errd_ej_et['Finess'].nunique()

#### % de valeurs manquantes par colonne

In [ ]:
est_une_valeur_manquante = ann_errd_ej_et.drop('Finess EJ', axis=1).set_index(['Finess', 'Année']).isna().reset_index()
établissement_avec_au_moins_une_valeur_renseignée = (~est_une_valeur_manquante.groupby(
    'Finess').any().drop('Année', axis=1))
établissement_avec_au_moins_une_valeur_renseignée.sum() / nombre_d_établissements_dans_ann_ms_tdp_et

In [ ]:
ann_errd_ej_et.groupby('Finess').any().sum() / nombre_d_établissements_dans_ann_ms_tdp_et


0 équivaut à valeur manquante ? Dans quels cas ?
Non, sauf si toute la ligne est à 0 (établissement fermé, sous CA)

### Cohérence des données

#### Dépenses positives

In [ ]:
(ann_errd_ej_et['Dépenses Groupe I ERRD'] > 0).sum()

In [ ]:
(ann_errd_ej_et['Dépenses Groupe II ERRD'] > 0).sum()
ann_errd_ej_et[ann_errd_ej_et['Dépenses Groupe II ERRD'] > 0]

In [ ]:
(ann_errd_ej_et['Dépenses Groupe III ERRD'] > 0).sum()
ann_errd_ej_et[ann_errd_ej_et['Dépenses Groupe III ERRD'] > 0]


#### Recettes négatives


In [ ]:
(ann_errd_ej_et['Recettes Groupe I ERRD'] < 0).sum()

In [ ]:
(ann_errd_ej_et['Recettes Groupe II ERRD'] < 0).sum()


In [ ]:
ann_errd_ej_et[ann_errd_ej_et['Recettes Groupe II ERRD'] < 0]

In [ ]:
(ann_errd_ej_et['Recettes Groupe III ERRD'] < 0).sum()


In [ ]:
ann_errd_ej_et[ann_errd_ej_et['Recettes Groupe III ERRD'] < 0]


#### Résultat

In [ ]:
résultat = ann_errd_ej_et['Recettes Groupe I ERRD'] + ann_errd_ej_et['Recettes Groupe II ERRD'] + ann_errd_ej_et[
    'Recettes Groupe III ERRD'] + ann_errd_ej_et['Dépenses Groupe I ERRD'] + ann_errd_ej_et['Dépenses Groupe II ERRD'] + ann_errd_ej_et['Dépenses Groupe III ERRD']


In [ ]:
résultat.name = 'résultat'

In [ ]:
ann_errd_ej_et.head()

In [ ]:
a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes = ~(résultat.round(decimals=2) == ann_errd_ej_et['MS Résultat net comptable ERRD'].round(decimals=2))
ann_errd_ej_et.loc[a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes, ["Finess",
                                                                                  "Année",
                                                                                  "MS Résultat net comptable ERRD"]].isna().sum()


In [ ]:
a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes.sum()

In [ ]:
ann_errd_ej_et.loc[a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes, ["Finess",
                                                                                  "Année",
                                                                                  "MS Résultat net comptable ERRD"]]

Les fois où le résultat n'est pas consistant avec la somme des recettes et des dépenses c'est parce qu'on n'a pas de valeur renseignée

#### 655 Quotes-parts de résultat sur opérations faites en commun


C'est un indicateur qui est positif sur notre maquette

In [ ]:
ann_errd_ej_et['655 Quotes-parts de résultat sur opérations faites en commun'].hist(bins=50)


In [ ]:
ann_errd_ej_et['655 Quotes-parts de résultat sur opérations faites en commun'].max()


In [ ]:
(ann_errd_ej_et['655 Quotes-parts de résultat sur opérations faites en commun'] > 0).sum()


In [ ]:
ann_errd_ej_et[ann_errd_ej_et['655 Quotes-parts de résultat sur opérations faites en commun'] > 0]


Est-ce normal d'avoir des valeurs positives ?
A priori non, c'est une charge des établissements donc négative


### Stats descriptives


In [ ]:
ann_errd_ej_et.describe()

In [ ]:
ann_errd_ej_et[[
    'Dépenses Groupe II ERRD',
    'Dépenses Groupe III ERRD',
    'Recettes Groupe I ERRD',
    'Recettes Groupe II ERRD',
    'Recettes Groupe III ERRD',
    'MS Résultat net comptable ERRD',
    '655 Quotes-parts de résultat sur opérations faites en commun']].hist(bins=25)

## ANN_CA_EJ_ET et ANN_ERRD_EJ_ET sont-ils bien disjoints ?


In [ ]:
finess_ann_ca_ej_et = set(ann_ca_ej_et['Finess'])

In [ ]:
finess_ann_errd_ej_et = set(ann_errd_ej_et['Finess'])

In [ ]:
len(finess_ann_ca_ej_et.intersection(finess_ann_errd_ej_et))

In [ ]:
ann_ca_ej_et.merge(ann_errd_ej_et, on=['Finess', 'Année'], how='inner')


In [ ]:
ann_errd_ej_et[[
    'Finess',
    'Année',
    'Dépenses Groupe II ERRD',
    'Dépenses Groupe III ERRD',
    'Recettes Groupe I ERRD',
    'Recettes Groupe II ERRD',
    'Recettes Groupe III ERRD',
    'MS Résultat net comptable ERRD']].merge(
    ann_ca_ej_et[
        [
            'Finess',
            'Année',
            'Dépenses Groupe II CA',
            'Dépenses Groupe III CA',
            'Recettes Groupe I CA',
            'Recettes Groupe II CA',
            'Recettes Groupe III CA',
            'MS Résultat net comptable CA PA',
            'MS Résultat net comptable CA PH']
    ],
    on=['Finess', 'Année'],
    how='inner'
)

Que fait-on pour les établissements que l'on retrouve à la fois dans la catégorie CA et ERRD ?
On privilégie les données issues de la catégorie ERRD

## ANN_CA_EJ_ET

### On s'attend à avoir une seule valeur par (Finess, Année)

In [ ]:
a_plus_d_une_donnée_par_finess_par_année = (ann_ca_ej_et.groupby(['Finess', 'Année']).nunique() > 1).any(axis=1)

In [ ]:
a_plus_d_une_donnée_par_finess_par_année.sum()

## Valeurs manquantes


In [ ]:
ann_ca_ej_et[ann_ca_ej_et["Charges CA PA"].isna() == False]

In [ ]:
ann_ca_ej_et.isna().sum() / ann_ca_ej_et.shape[0]

In [ ]:
aucune_valeur_de_résultat = (ann_ca_ej_et["MS Résultat net comptable CA PA"].isna() & ann_ca_ej_et[
    "MS Résultat net comptable CA PH"].isna())
aucune_valeur_de_résultat.sum()

In [ ]:
ann_ca_ej_et[aucune_valeur_de_résultat]

### Années renseignées

In [ ]:
ann_ca_ej_et.groupby('Finess').count()['Année'].hist()


In [ ]:
ann_ca_ej_et['Année'].nunique()

In [ ]:
ann_ca_ej_et['Année'].hist()

In [ ]:
ann_ca_ej_et['Année'].max()

Dans l'autre fichier, on observait la tendance inverse : plus d'années récentes, moins d'années passées

### Valeurs manquantes par colonne

In [ ]:
nombre_d_établissements_dans_ann_ms_tdp_et = ann_ca_ej_et['Finess'].nunique()

#### % de valeurs manquantes par colonne

In [ ]:
est_une_valeur_manquante = ann_ca_ej_et.drop('Finess EJ', axis=1).set_index(['Finess', 'Année']).isna().reset_index()
établissement_avec_au_moins_une_valeur_renseignée = (~est_une_valeur_manquante.groupby(
    'Finess').any().drop('Année', axis=1))
établissement_avec_au_moins_une_valeur_renseignée.sum() / nombre_d_établissements_dans_ann_ms_tdp_et

On a beaucoup plus d'établissements pour lesquels le résultat est calculé en mode PA que PH

In [ ]:
ann_ca_ej_et.groupby('Finess').any().sum() / nombre_d_établissements_dans_ann_ms_tdp_et


0 équivaut à valeur manquante ? Dans quels cas ?

Que faire lorsque les recettes sont calculées en mode "Charges / Produits" ? On n'a pas la maquette

### Cohérence des données

#### Dépenses positives

In [ ]:
(ann_ca_ej_et['Dépenses Groupe I CA'] > 0).sum()
ann_ca_ej_et[ann_ca_ej_et['Dépenses Groupe I CA'] > 0]

In [ ]:
(ann_ca_ej_et['Dépenses Groupe II CA'] > 0).sum()

In [ ]:
(ann_ca_ej_et['Dépenses Groupe III CA'] > 0).sum()
ann_ca_ej_et[ann_ca_ej_et['Dépenses Groupe III CA'] > 0]

In [ ]:
(ann_ca_ej_et['Charges CA PA'] > 0).sum()
ann_ca_ej_et[ann_ca_ej_et['Charges CA PA'] > 0]

#### Recettes négatives


In [ ]:
(ann_ca_ej_et['Recettes Groupe I CA'] < 0).sum()

In [ ]:
(ann_ca_ej_et['Recettes Groupe II CA'] < 0).sum()


In [ ]:
ann_ca_ej_et[ann_ca_ej_et['Recettes Groupe II CA'] < 0]

In [ ]:
(ann_ca_ej_et['Recettes Groupe III CA'] < 0).sum()


In [ ]:
ann_ca_ej_et[ann_ca_ej_et['Recettes Groupe III CA'] < 0]


In [ ]:
(ann_ca_ej_et['Produits CA PA'] < 0).sum()
ann_ca_ej_et[ann_ca_ej_et['Produits CA PA'] < 0]

#### Résultat

In [ ]:
ann_ca_ej_et_avec_résultat_CA_PH_renseigné = ann_ca_ej_et.dropna(subset=['MS Résultat net comptable CA PH'])

In [ ]:
résultat = ann_ca_ej_et_avec_résultat_CA_PH_renseigné['Recettes Groupe I CA'] + ann_ca_ej_et_avec_résultat_CA_PH_renseigné['Recettes Groupe II CA'] + ann_ca_ej_et_avec_résultat_CA_PH_renseigné[
    'Recettes Groupe III CA'] + ann_ca_ej_et_avec_résultat_CA_PH_renseigné['Dépenses Groupe I CA'] + ann_ca_ej_et_avec_résultat_CA_PH_renseigné['Dépenses Groupe II CA'] + ann_ca_ej_et_avec_résultat_CA_PH_renseigné['Dépenses Groupe III CA']


In [ ]:
résultat.name = 'résultat'

In [ ]:
a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes = ~(résultat.round(decimals=2) == ann_ca_ej_et_avec_résultat_CA_PH_renseigné['MS Résultat net comptable CA PH'].round(decimals=2))
ann_ca_ej_et_avec_résultat_CA_PH_renseigné.loc[a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes, ["Finess",
                                                                                  "Année",
                                                                                  "MS Résultat net comptable CA PH"]].isna().sum()


In [ ]:
ann_ca_ej_et_avec_résultat_CA_PA_renseigné = ann_ca_ej_et.dropna(subset=['MS Résultat net comptable CA PA'])

In [ ]:
résultat = ann_ca_ej_et_avec_résultat_CA_PA_renseigné['Produits CA PA'] + ann_ca_ej_et_avec_résultat_CA_PA_renseigné['Charges CA PA']

In [ ]:
a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes = ~(résultat.round(decimals=2) == ann_ca_ej_et_avec_résultat_CA_PA_renseigné['MS Résultat net comptable CA PA'].round(decimals=2))
ann_ca_ej_et_avec_résultat_CA_PA_renseigné.loc[a_un_résultat_différent_que_la_somme_des_dépenses_et_recettes, ["Finess",
                                                                                                               "Année",
                                                                                                               "MS Résultat net comptable CA PA"]].isna().sum()


### Taux de vétusté construction

In [ ]:
a_une_vétusté_négative = ann_ca_ej_et['Taux vétusté Construction CA'] < 0
(a_une_vétusté_négative).sum()

In [ ]:
ann_ca_ej_et[a_une_vétusté_négative]

In [ ]:
a_une_vétusté_sup_à_1 = ann_ca_ej_et['Taux vétusté Construction CA'] > 1
(a_une_vétusté_sup_à_1).sum()

In [ ]:
ann_ca_ej_et[a_une_vétusté_sup_à_1]

Que fait-on de taux non compris entre 0 et 1 ?
Si > 1, alors affichage en rouge

### Taux de CAF

Question pour Nicolas : quelles sont les valeurs normales ?
C'est plutôt négatif. De l'ordre de 2% (seuil)

In [ ]:
a_une_caf_négative = ann_ca_ej_et['Taux de CAF CA PH'] < 0
(a_une_caf_négative).sum()

In [ ]:
ann_ca_ej_et[a_une_caf_négative]

In [ ]:
a_une_caf_sup_à_1 = ann_ca_ej_et['Taux de CAF CA PH'] > 1
(a_une_caf_sup_à_1).sum()

In [ ]:
ann_ca_ej_et[a_une_caf_sup_à_1]

Beaucoup de taux de CAF négatifs.

### Stats descriptives


In [ ]:
ann_ca_ej_et.describe()


## ANN_ERRD_EJ

In [ ]:
ann_ca_ej_et[['Finess EJ', 'Finess', 'Année','Taux de CAF CA PH', 'Taux vétusté Construction CA']].merge(
    ann_errd_ej[['Finess EJ', 'Année','Taux de CAF ERRD', 'Taux vétusté Construction ERRD']]
, on=['Finess EJ', 'Année'], how='inner')


Quelle règle de gestion lorsqu'on a l'info des deux côtés ?
On privilégie aussi l'ERRD

### Taux de vétusté

In [ ]:
a_une_vétusté_négative = ann_errd_ej['Taux vétusté Construction ERRD'] < 0
(a_une_vétusté_négative).sum()

In [ ]:
ann_errd_ej[a_une_vétusté_négative]

In [ ]:
a_une_vétusté_sup_à_1 = ann_errd_ej['Taux vétusté Construction ERRD'] > 1
(a_une_vétusté_sup_à_1).sum()

In [ ]:
ann_errd_ej[a_une_vétusté_sup_à_1]

Que fait-on de taux non compris entre 0 et 1 ?

### Taux de CAF

Question pour Nicolas : quelles sont les valeurs normales ?

In [ ]:
a_une_caf_négative = ann_errd_ej['Taux de CAF ERRD'] < 0
(a_une_caf_négative).sum()

In [ ]:
ann_errd_ej[a_une_caf_négative]

In [ ]:
a_une_caf_sup_à_1 = ann_errd_ej['Taux de CAF ERRD'] > 1
(a_une_caf_sup_à_1).sum()

In [ ]:
ann_errd_ej[a_une_caf_sup_à_1]

Beaucoup de taux de CAF négatifs.
